Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Demo 11 - Dashboards

- Retrieve and cache data from bikeshare every X minutes
- Map counts of available bikes per tract
- Use slider to move back and forth through time with available data
- Checkbox to identify tracts where values have changed since app started? (change color of bubbles?)
- Checkbox to select different types of bikes? (electric vs ebike?)
- Parameter to scale size of bubbles?

In [1]:
import demo11
import geopandas as gpd
import pandas as pd
import numpy as np
import json

%load_ext autoreload
%autoreload 2

In [9]:
# Load Data
bikes_gdf, stations_gdf = demo11.load_bike_data()
bikes_gdf = pd.concat([
    bikes_gdf[['lat','lon','vehicle_type_id','timestamp','geometry']],
    demo11.expand_stations_into_bikes(stations_gdf)])
tracts_gdf = demo11.load_tracts()
eea_df = pd.read_csv('mwcog_eea_2016_2020.csv')
eea_df = demo11.clean_eea(eea_df)
nbhds_gdf = demo11.load_nbhds()

# Join equity emphasis area data to tracts
tracts_gdf = tracts_gdf.merge(eea_df, on='tract_id', how='left')

# Add tract data to nbhds with weighted average eea indices
# Also limit to nhbds with tract data
nbhds_gdf = demo11.add_tract_data_to_nbhds(nbhds_gdf, tracts_gdf)

In [10]:
nbhds_gdf

,nbhd,geometry,pop,eea_idx,eea_low_income,eea_black,eea_hisp,eea_asian
0,Colonial Village,POINT (-77.04109 38.98679),4477.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,Takoma,POINT (-77.02156 38.97646),7481.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,Barnaby Woods,POINT (-77.06017 38.97543),5804.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Brightwood,POINT (-77.02687 38.96638),14012.0,7.909646,1.780667,1.461645,1.266913,0.032542
4,Lamond Riggs,POINT (-77.00645 38.96507),4286.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
105,Douglass,POINT (-76.97964 38.84757),8885.0,8.645612,2.429001,3.428605,0.313414,0.020046
106,Shipley,POINT (-76.9729 38.84686),6139.0,12.000000,4.622832,3.263451,0.293282,0.061169
107,Congress Heights,POINT (-76.99795 38.84108),10617.0,12.000000,3.365381,3.468561,0.202736,0.010956
108,Washington Highlands,POINT (-76.99564 38.83024),11484.0,12.000000,3.615912,3.683990,0.134923,0.009015


In [11]:
# Attach neighborhoods to bikes
bikes_gdf = demo11.attach_points_to_points(bikes_gdf, nbhds_gdf)

In [12]:
bikes_gdf

,lat,lon,vehicle_type_id,timestamp,geometry,station_id,nbhd,pop,eea_idx,eea_low_income,eea_black,eea_hisp,eea_asian
0,38.860617,-77.067196,2,2025-04-14 17:36:45,POINT (-77.0672 38.86062),NaN,Monumental Core,17.0,0.000000,0.000000,0.000000,0.000000,0.00000
1,38.859086,-76.997106,2,2025-04-14 17:36:45,POINT (-76.99711 38.85909),NaN,Barry Farm,1407.0,12.000000,5.635000,3.415000,0.201000,0.25700
2,38.907483,-76.951222,2,2025-04-14 17:36:45,POINT (-76.95122 38.90748),NaN,Mayfair,5144.0,12.000000,4.398000,3.439000,0.548000,0.02400
3,38.831194,-77.118111,2,2025-04-14 17:36:45,POINT (-77.11811 38.83119),NaN,Georgetown Reservoir,3306.0,0.000000,0.000000,0.000000,0.000000,0.00000
4,38.929028,-77.004184,2,2025-04-14 17:36:45,POINT (-77.00418 38.92903),NaN,Stronghold,3337.0,7.510000,1.641000,2.593000,0.613000,0.08300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,38.990249,-77.029350,1,2025-04-14 17:36:28,POINT (-77.02935 38.99025),08259804-1f3f-11e7-bf6b-3863bb334450,Colonial Village,4477.0,0.000000,0.000000,0.000000,0.000000,0.00000
5233,38.990249,-77.029350,1,2025-04-14 17:36:28,POINT (-77.02935 38.99025),08259804-1f3f-11e7-bf6b-3863bb334450,Colonial Village,4477.0,0.000000,0.000000,0.000000,0.000000,0.00000
5234,38.990249,-77.029350,2,2025-04-14 17:36:28,POINT (-77.02935 38.99025),08259804-1f3f-11e7-bf6b-3863bb334450,Colonial Village,4477.0,0.000000,0.000000,0.000000,0.000000,0.00000
5235,38.876941,-77.017807,1,2025-04-14 17:36:29,POINT (-77.01781 38.87694),08247896-1f3f-11e7-bf6b-3863bb334450,Southwest/Waterfront,9397.0,2.200084,0.733642,0.466519,0.220429,0.13251


In [ ]:
tract_points_by_neighborhood[tract_points_by_neighborhood.nbhd == 'North Portal Estates']

In [ ]:
nbhds_gdf

In [ ]:
eea_df[eea_df.tract_id.str.endswith('3600')]

In [ ]:
tracts_gdf[tracts_gdf.tract_id.str.endswith('3600')]

In [ ]:
tracts_gdf = tracts_gdf.merge(eea_df, on='tract_id', how='left')

In [ ]:
len(tracts_gdf)

In [ ]:
tracts_gdf

In [ ]:
# Retrieve bike data and join with tracts
tracts_gdf = demo11.load_tracts()
bikes_gdf, stations_gdf = demo11.load_bike_data()
bikes_gdf = pd.concat([
    bikes_gdf[['lat','lon','vehicle_type_id','timestamp','geometry']],
    demo11.expand_stations_into_bikes(stations_gdf)])

In [ ]:
neighborhood_points_gdf = gpd.read_file('dc_neighborhood_points.geojson')
neighborhood_points_gdf = neighborhood_points_gdf[['NAME','geometry']].rename(columns={'NAME':'nbhd'})

In [ ]:
bikes_gdf.to_crs(demo11.UTM18)

In [ ]:


gpd.sjoin_nearest(bikes_gdf.to_crs(demo11.UTM18), neighborhood_points_gdf.to_crs(demo11.UTM18))

In [ ]:
bikes_gdf = demo11.attach_tracts_to_bikes(bikes_gdf, tracts_gdf)

# Filter bikes based on ui criteria (electric/non-electric; docked/undocked)

# Count bikes per tract for mapping
bikes_per_tract_df = demo11.count_bikes_per_tract(bikes_gdf)
tracts_with_bike_counts_gdf = demo11.attach_counts_to_tract_polygons(bikes_per_tract_df, tracts_gdf)
tracts_with_bike_counts_gdf = demo11.polygons_to_points(tracts_with_bike_counts_gdf)

In [ ]:
tracts_with_bike_counts_gdf[tracts_with_bike_counts_gdf.geometry.isnull()]

In [ ]:
bikes_gdf.groupby('tract_id').size().rename('bikes')

In [ ]:
bikes_per_tract_df[bikes_per_tract_df.bikes.isnull()]

In [ ]:
bikes_gdf.vehicle_type_id.value_counts()

In [ ]:
bikes_per_tract_df['at_station'] = bikes_per_tract_df['station_id'].astype(bool)

In [ ]:
bikes_gdf.station_id.notnull().astype(int)

In [ ]:
pd.DataFrame(bikes_per_tract_df.groupby('tract_id').size().rename('bikes'))

In [ ]:
multiplier = 100
for tract in tract_points_with_bike_counts_gdf.itertuples():
    # break
    circle_marker = CircleMarker()
    circle_marker.location = (tract.geometry.y, tract.geometry.x)
    circle_marker.radius = tract.available_bikes * multiplier
    circle_marker.color = "red"
    circle_marker.fill_color = "red"

In [ ]:
tract

In [ ]:
tracts_with_bike_count_circles_gdf = demo11.replace_polygons_with_proportional_circles(
    tracts_with_bike_counts_gdf, 'available_bikes', 250)

In [ ]:
tracts_with_bike_count_circles_gdf = tracts_with_bike_count_circles_gdf.to_crs(4326)

In [ ]:
tracts_with_bike_count_circles_gdf[['available_bikes','geometry']].to_geo_dict()

In [ ]:
from shiny import render, reactive
from shiny.express import input, render, ui
from shinywidgets import render_widget
import ipywidgets as widgets

In [ ]:
city_centers = {
    "London": (51.5074, 0.1278),
    "Paris": (48.8566, 2.3522),
    "New York": (40.7128, -74.0060)
}

ui.input_select("center", "Center", choices=list(city_centers.keys()))

@render_widget
def map():
    return ipyl.Map(zoom=4)

@reactive.effect
def _():
    map.widget.center = city_centers[input.center()]

In [ ]:
ui.input_text("name", "What's your name?", value="World")

@render.text
def greeting():
    return f"Hello, {input.name()}!"

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## GeoPandas `plot`

The simplest option for making a static map from a GeoDataFrame is the built-in `plot` method.

In [ ]:
crs = 26918 # UTM18
affordable_housing = gpd.read_file('affordable_housing.geojson').to_crs(crs)
tracts = gpd.read_file('dc_tracts.geojson').to_crs(crs)
# Restrict to tract-level features, which have geoids 18 characters long
tracts = tracts[tracts.geoid.str.len() == 18]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax = tracts.plot(ax=ax, facecolor='none', edgecolor='k', zorder=0)
affordable_housing.plot(ax=ax, markersize=5, color='r', zorder=1)
ax.axis('off')

plt.savefig('large_scale_map.eps')

plt.show()

In [ ]:
affordable_housing = affordable_housing.sjoin(tracts[['geoid','geometry']])

In [ ]:
tracts = tracts.set_index('geoid')

In [ ]:
tracts = pd.concat([tracts, affordable_housing.groupby('geoid').agg({'TOTAL_AFFORDABLE_UNITS':'sum'})], axis=1)
tracts = tracts.fillna(0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax = tracts.plot(ax=ax, column='TOTAL_AFFORDABLE_UNITS', cmap='PuBu', linewidth=0.5, edgecolor='black', legend=True)
affordable_housing.plot(ax=ax, markersize=5, color='r', zorder=1)
ax.axis('off')

plt.savefig('large_scale_map.eps')

plt.show()

In [ ]:
tracts['affordable_units_per_pop'] = tracts['TOTAL_AFFORDABLE_UNITS'] / tracts['B01003001']

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax = tracts.plot(ax=ax, column='affordable_units_per_pop', cmap='PuBu', linewidth=0.5, edgecolor='black', legend=True)
affordable_housing.plot(ax=ax, markersize=5, color='r', zorder=1)
ax.axis('off')

plt.savefig('large_scale_map.eps')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax = tracts.plot(ax=ax, column='TOTAL_AFFORDABLE_UNITS', cmap='PuBu', linewidth=0.5, edgecolor='black', legend=True)
affordable_housing.plot(ax=ax, markersize=5, color='r', zorder=1)
ax.axis('off')

plt.savefig('large_scale_map.eps')

plt.show()

### Proportional Symbol Map

An elegant way to map counts without biasing by the areas of spatial units is to use proportional symbols. This script calculates the radii (buffer distances) for circular symbols with areas proportional to input counts. 

In [ ]:
tract_points = tracts.copy()
tract_points['geometry'] = tract_points.centroid

def proportional_circles_radii(values, multiplier=1):
    return np.sqrt(values / 3.14) * multiplier

fig, ax = plt.subplots(figsize=(10, 10))
ax = tracts.plot(ax=ax, facecolor='none', edgecolor='black', zorder=0, linewidth=0.5)

tract_points.buffer(proportional_circles_radii(tract_points['TOTAL_AFFORDABLE_UNITS'], 25)).plot(
    ax=ax, 
    zorder=1, 
    facecolor='lavender', 
    edgecolor='midnightblue', 
    linewidth=0.5)
# affordable_housing.plot(ax=ax, markersize=1, color='darkblue', zorder=2)
ax.axis('off')

plt.savefig('large_scale_map.eps')

plt.show()

## Contextly can automatically add a basemap for reference

In [ ]:
# !pip install contextily

In [ ]:
import contextily as cx

In [ ]:
tract_points = tracts.copy()
tract_points['geometry'] = tract_points.centroid

def proportional_circles_radii(values, multiplier=1):
    return np.sqrt(values / 3.14) * multiplier

fig, ax = plt.subplots(figsize=(10, 10))
ax = tracts.plot(ax=ax, facecolor='none', edgecolor='k', zorder=1, linewidth=0.5)

tract_points.buffer(proportional_circles_radii(tract_points['TOTAL_AFFORDABLE_UNITS'], 25)).plot(
    ax=ax, 
    facecolor='lavender', 
    edgecolor='midnightblue', 
    linewidth=0.5, 
    zorder=2)
ax.axis('off')

# Base map with labels embedded
cx.add_basemap(ax, crs=crs, source=cx.providers.CartoDB.Positron, zorder=0)

# Base map with labels overlaid
# cx.add_basemap(ax, crs=crs, source=cx.providers.CartoDB.PositronNoLabels, zorder=0)
# cx.add_basemap(ax, crs=crs, source=cx.providers.CartoDB.PositronOnlyLabels, zorder=3)

plt.savefig('large_scale_map.eps')

plt.show()

## Folium

To make an interactive map with the popular [Leaflet.js](https://leafletjs.com/reference) JavaScript library, we can use Folium.

In [ ]:
# !pip install folium

In [ ]:
import folium

In [ ]:
affordable_housing = gpd.read_file('affordable_housing.geojson')
tracts = gpd.read_file('dc_tracts.geojson')

# Project into lat and long (WGS84)
affordable_housing = affordable_housing.to_crs(4326)
tracts = tracts.to_crs(4326)

# Drop column with timestamps because they interfere with Folium
affordable_housing = affordable_housing.drop(columns=['GIS_LAST_MOD_DTTM'])

# Identify map center
lon_min, lat_min, lon_max, lat_max = tracts.total_bounds
lon_center = (lon_max + lon_min) / 2
lat_center = (lat_max + lat_min) / 2

# Initiate map
map = folium.Map(location=(lat_center, lon_center), zoom_start=12)
# map = folium.Map(location=(lat_center, lon_center), zoom_start=12, tiles="Cartodb Positron")

# Add tracts
tract_style = {
    'weight': 1,
    'color': 'black',
    'fillColor': '#00000000',
    'fillOpacity': 1,
    }
folium.GeoJson(data=tracts.to_json(), style_function=lambda x: tract_style).add_to(map)

# Add affordable housing points
circle_marker = folium.CircleMarker(radius = 4, weight = 1, color='white', fillColor = 'darkblue', fill_opacity=1)
affordable_housing_points = folium.GeoJson(affordable_housing.to_json(), marker=circle_marker)
folium.features.GeoJsonPopup(fields=['PROJECT_NAME','TOTAL_AFFORDABLE_UNITS'], labels=True).add_to(affordable_housing_points)
affordable_housing_points.add_to(map)

map

## GeoPandas `explore`

To make life just a little easier, GeoPandas has a built-in method for making a Folium map with `explore`.

In [ ]:
m = tracts.explore(
    tiles='CartoDB positron',
    attr='MapBox CartoDB positron',
    style_kwds={
        'weight': 1,
        'color': 'black',
        'fillColor': '#00000000',
        'fillOpacity': 1,
    },
)

m = affordable_housing.explore(
    m=m,
    marker_kwds={
        'radius':4, 
        'weight':1, 
        'color':'white', 
        'fillColor':'darkblue', 
        'fill_opacity':1
    },
)

m

## CartoPy

If you're making a smaller-scale map of a state, country, or continent (can you guess why the scale is called "smaller"?) you may need to use a specialized projection to avoid distortion or preserve particular qualities (e.g., using an equidistant projection to allow for measurement between distant points).

A package called CartoPy enables mapping in Python with nearly any projection. It also allows you to access convenient base layers from [Natural Earth](https://www.naturalearthdata.com/), a free database of global basemap data for cultural (e.g., borders) and physical (e.g., coastlines) features.

In [ ]:
# !pip install cartopy

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
# Define highlight city
highlight_city = 'Washington,  D.C.'
# highlight_city = 'Chicago'

# Load city points
pop_places = gpd.read_file('ne_10m_populated_places_simple.zip')

# Define map center and extents around the highlight city
lon_center = pop_places[pop_places['name'] == highlight_city].geometry.x.iloc[0]
lat_center = pop_places[pop_places['name'] == highlight_city].geometry.y.iloc[0]
extent = 50
lon_min = lon_center + (extent / 2)
lon_max = lon_center - (extent / 2)
lat_min = lat_center - (extent / 2)
lat_max = lat_center + (extent / 2)

# Define projection centered on highlight city
crs = ccrs.AzimuthalEquidistant(central_longitude=lon_center, central_latitude=lat_center)

# Project cities into axis coordinate system
pop_places = pop_places.to_crs(crs.proj4_init)

# Constrain to cities only one rank lower than the highlight
rank = pop_places[pop_places['name'] == highlight_city].scalerank.iloc[0] + 1
pop_places = pop_places[pop_places.scalerank <= rank]

# Initiate figure
fig = plt.figure(figsize=(15, 10))

# Initiate axis with defined projection
ax = plt.subplot(1, 1, 1, projection=crs)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add base features
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='blue', linestyle='--', zorder=0)
ax.add_feature(cfeature.LAND, zorder=1)
ax.add_feature(cfeature.COASTLINE, zorder=2)
ax.add_feature(cfeature.LAKES, alpha=0.5, zorder=3)
ax.add_feature(cfeature.STATES, zorder=4)
ax.add_feature(cfeature.RIVERS, zorder=5)

# Add city points
pop_places.plot(ax=ax, markersize=5, color='k', zorder=1)
pop_places[pop_places['name'] == highlight_city].plot(ax=ax, markersize=50, color='r', zorder=6)

# Label cities
def annotate(record):
    if record['name'] == highlight_city:
        ax.annotate(record['name'], weight='bold', size=14, color='red', xy=record.geometry.coords[0], xytext=(3, 3), textcoords='offset points', clip_on=True)
    else:
        ax.annotate(record['name'], xy=record.geometry.coords[0], xytext=(3, 3), textcoords='offset points', clip_on=True)
pop_places.apply(annotate, axis=1)

plt.savefig('small_scale_map.eps', format='eps')

plt.show()